In [ ]:
1.	import csv
2.	from langchain_core.documents import Document
3.	from langchain_openai import OpenAIEmbeddings, ChatOpenAI
4.	from langchain_community.vectorstores import FAISS
5.	from langchain.text_splitter import RecursiveCharacterTextSplitter
6.	from langchain.chains import RetrievalQA
7.
8.	# Step 1: Load CSV file
9.	csv_path = "customer_support_tickets.csv"
10.	documents = []
11.
12.	with open(csv_path, newline='', encoding='utf-8') as csvfile:
13.	   reader = csv.DictReader(csvfile)
14.	   for index, row in enumerate(reader):
15.	       # Build the content from subject, description, and resolution
16.	       content = f"""Customer Issue: {row['Ticket Subject']}\nDescription: {row['Ticket Description']}\nSupport Response: {row['Resolution']}"""
17.	       documents.append(Document(page_content=content, metadata={"source": f"ticket_{index}"}))
18.
19.	# Step 2: Chunking
20.	splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
21.	chunks = splitter.split_documents(documents)
22.
23.	# Step 3: Generate Embeddings
24.	embeddings = OpenAIEmbeddings(openai_api_key="API-KEY")
25.	vectorstore = FAISS.from_documents(chunks, embeddings)
26.	retriever = vectorstore.as_retriever()
27.
28.	# Step 4: Use an LLM model
29.	llm = ChatOpenAI(model_name="gpt-4o", temperature=0.2, openai_api_key="API-KEY")
30.
31.	# Step 5: Use a RAG chain
32.	rag_chain = RetrievalQA.from_chain_type(
33.	   llm=llm,
34.	   chain_type="stuff",
35.	   retriever=retriever,
36.	   return_source_documents=True
37.	)
38.
39.	# Step 6: Ask a query
40.	query = "What should I do if my software crashes?"
41.	result = rag_chain.invoke(query)
42.
43.	print("\nAnswer:\n", result['result'])
44.	print("\nSources:")
45.	for doc in result['source_documents']:


In [ ]:
1.	import csv
2.	from langchain.chains import LLMChain
3.	from langchain.prompts import PromptTemplate
4.	from langchain_openai import ChatOpenAI
5.
6.	# Step 1: load the dataset (reviews_dataset.csv)
7.	csv_path = "reviews_dataset.csv"
8.	comments = []
9.
10.	with open(csv_path, newline='', encoding='cp1252') as csvfile:
11.	   reader = csv.DictReader(csvfile)
12.	   for row in reader:
13.	       if row.get("Comments"):
14.	           comments.append(row["Comments"])
15.
16.	# Step 2: set up the LLM
17.	llm = ChatOpenAI(model_name="gpt-4o", temperature=0.2, openai_api_key="api key")
18.
19.	# Step 3: define a prompt that extracts polarity
20.	polarity_prompt = PromptTemplate(
21.	   input_variables=["review"],
22.	   template="""
23.	   Analyze the sentiment polarity (Positive, Neutral, or Negative) of the following customer review:
24.	   Review: {review}
25.	   """
26.	)
27.
28.	# Step 3: define a prompt that extracts aspect-based sentiment (product, service, delivery)
29.	aspect_prompt = PromptTemplate(
30.	   input_variables=["review"],
31.	   template="""
32.	   Analyze this customer review and extract sentiment for the following aspects: Product, Service, and Delivery.
33.	   Return the aspect sentiments in this format:
34.	   Product: [Positive/Neutral/Negative]
35.	   Service: [Positive/Neutral/Negative]
36.	   Delivery: [Positive/Neutral/Negative]
37.
38.	   Review: {review}
39.	   """
40.	)
41.
42.	# Step 4: build the prompt chains
43.	polarity_chain = polarity_prompt | llm
44.	aspect_chain = aspect_prompt | llm
45.
46.	# Step 5: run the analysis, and substitute the review argument
47.	for index, comment in enumerate(comments, 1):
48.	   print(f"\n--- Review #{index} ---")
49.	   print("Original Comment:", comment)
50.
51.	   polarity = polarity_chain.invoke({"review": comment}).content.strip()
52.	   aspects = aspect_chain.invoke({"review": comment}).content.strip()
53.
54.	   print("Polarity:", polarity)
55.	   print("Aspect-Based Sentiment:\n", aspects)


In [ ]:
1.	from langchain_core.documents import Document
2.	from langchain.text_splitter import RecursiveCharacterTextSplitter
3.	from langchain_community.vectorstores import FAISS
4.	from langchain_openai import OpenAIEmbeddings, ChatOpenAI
5.	from langchain.chains import RetrievalQA
6.
7.	# Step 1: Load the policy document
8.	with open("policy.txt", "r", encoding="utf-8") as f:
9.	   policy_text = f.read()
10.
11.	documents = [Document(page_content=policy_text)]
12.
13.	# Step 2: Chunk the policy document and create embeddings.
14.	splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
15.	chunks = splitter.split_documents(documents)
16.
17.	# Step 3: create the vector index
18.	embeddings = OpenAIEmbeddings(openai_api_key="api-key")
19.	vectorstore = FAISS.from_documents(chunks, embeddings)
20.	retriever = vectorstore.as_retriever()
21.
22.	# Step 4: set up the LLM
23.	llm = ChatOpenAI(model_name="gpt-4o", temperature=0.3, openai_api_key="api-key")
24.
25.	# Step 5: Set up the RAG system
26.	rag_chain = RetrievalQA.from_chain_type(
27.	   llm=llm,
28.	   chain_type="stuff",  # simple concatenation of retrieved policy text
29.	   retriever=retriever,
30.	   return_source_documents=True
31.	)
32.
33.	# Step 6: Show an example of handling a complaint
34.	complaint = "My sushi arrived cold and 45 minutes late. Can I get a refund?"
35.
36.	result = rag_chain.invoke(complaint)
37.
38.	# Step 7: Show resolution and evidence
39.	print("\nResolution:\n", result["result"])
40.	print("\nPolicy Evidence Used:")
41.	for doc in result['source_documents']:
42.	  print(f"- {doc.page_content[:150]}...")